In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import gzip
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import time
import math

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)
print('ok')

ok


In [2]:
cnt=0
for l in readGz("data/train.json.gz"):
    print(l)
    cnt+=1
    if cnt>=5:
        break

{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}
{'rating': 5.0, 'businessID': 'B410583770', 'reviewText': "Quaint, stylish atmosphere in a small place right off South Congress. The food and service were great. It's pricey, but the happy hour deals are considerable. Great for a date!", 'userID': 'U637588426', 'reviewTime': 'Sep 21, 2011', 'categories': ['Italian Restaurant', 'European Restaurant'], 'reviewHash': 'R954562590', 'unixReviewTime': 1316616175}
{'rating': 4.0, 'businessID': 'B423621081', 'reviewText': "I'm a vegetarian, but every so often I want a hotdog with lots of toppings.  And a tall can of beer.  F

In [3]:
users=[]
bussiness=[]
ratings=[]
for l in readGz("data/train.json.gz"):
    u=l['userID']
    b=l['businessID']
    r=float(l['rating'])
    users.append(u)
    bussiness.append(b)
    ratings.append(r)
    
user_set=set(users)
buss_set=set(bussiness)

Nu=len(user_set)  # 18793
Nb=len(buss_set)  # 21321

user_index={}
buss_index={}

for (i,u) in enumerate(user_set):
    user_index[u]=i
for (i,b) in enumerate(buss_set):
    buss_index[b]=i
print(Nu,Nb)
print('ok')

18793 21321
ok


In [4]:
num_train=180000

y=np.zeros((Nu,Nb),dtype=np.bool_)

train_set=[]
val_set=[]

user_bussiness={}
ub_rating={}

n_train=0
n_val=0

total_rating_train=0

for i in range(len(users)):
    u=user_index[users[i]]
    b=buss_index[bussiness[i]]
    r=ratings[i]
    if i<num_train:
        y[u,b]=1
        total_rating_train+=r
        train_set.append((u,b,r))
        if u in user_bussiness:
            user_bussiness[u].append((b,r))
        else:
            user_bussiness[u]=[(b,r)]
        ub_rating[(u,b)]=r
        n_train+=1
    else:
        val_set.append((u,b,r))
        n_val+=1
        
for i in range(Nu):
    if i not in user_bussiness:
        user_bussiness[i]=[]

total_rating_train/=num_train
        
print('ok')


ok


In [5]:
avg_rating={}
for (u,b_r_list) in user_bussiness.items():
    tot_r=0
    for (b,r) in b_r_list:
        tot_r+=r
    if len(b_r_list)>0:
        tot_r/=len(b_r_list)
    avg_rating[u]=tot_r

In [6]:
def pearson_sim(u1,u2):
    new_u1=u1
    new_u2=u2
    if len(user_bussiness[u1])>len(user_bussiness[u2]):
        new_u1=u2
        new_u2=u1
    numerator=0
    denominator1=0
    denominator2=0
    for (b,r) in user_bussiness[new_u1]:
        if y[new_u2,b]==1:
            numerator+=(r-avg_rating[new_u1])*(ub_rating[(new_u2,b)]-avg_rating[new_u2])
            denominator1+=(r-avg_rating[new_u1])**2
            denominator2+=(ub_rating[(new_u2,b)]-avg_rating[new_u2])**2
    if denominator1==0 or denominator2==0:
        return 0
    else:
        return numerator/math.sqrt(denominator1*denominator2)

In [7]:
sim=np.zeros((Nu,Nu))
for i in range(Nu):
    for j in range(i):
        sim[i,j]=sim[j,i]=pearson_sim(i,j)
    if i==0 or (i+1)%1000==0 or i==Nu-1:
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),'finished processing %d users' %(i+1))

2017-11-19 18:47:53 finished processing 1 users
2017-11-19 18:47:57 finished processing 1000 users
2017-11-19 18:48:11 finished processing 2000 users
2017-11-19 18:48:32 finished processing 3000 users
2017-11-19 18:49:00 finished processing 4000 users
2017-11-19 18:49:35 finished processing 5000 users
2017-11-19 18:50:15 finished processing 6000 users
2017-11-19 18:51:03 finished processing 7000 users
2017-11-19 18:52:00 finished processing 8000 users
2017-11-19 18:53:08 finished processing 9000 users
2017-11-19 18:54:29 finished processing 10000 users
2017-11-19 18:56:01 finished processing 11000 users
2017-11-19 18:57:41 finished processing 12000 users
2017-11-19 18:59:29 finished processing 13000 users
2017-11-19 19:01:24 finished processing 14000 users
2017-11-19 19:03:32 finished processing 15000 users
2017-11-19 19:05:49 finished processing 16000 users
2017-11-19 19:08:10 finished processing 17000 users
2017-11-19 19:10:41 finished processing 18000 users
2017-11-19 19:12:48 finis

In [36]:
def nearest_k_user(u,K):
    index=list(np.argsort(sim[u,:]))
    return reversed(index[-(K+1):-1])

In [43]:
u=10
u_list=nearest_k_user(u,10)
for un in u_list:
    print(un,sim[u,un])

1020 1.0
10849 1.0
8460 1.0
17978 1.0
9269 1.0
6281 1.0
8999 0.902134221636
13740 0.640184399664
6265 0.0
6266 0.0


In [45]:
print(avg_rating[10])
print(avg_rating[10849])
print(user_bussiness[10])
print(user_bussiness[10849])

3.75
4.1
[(8368, 3.0), (12403, 4.0), (9559, 4.0), (17991, 4.0)]
[(4553, 4.0), (1284, 4.0), (19751, 4.0), (12247, 4.0), (17260, 4.0), (8368, 3.0), (11181, 4.0), (17013, 4.0), (340, 5.0), (160, 5.0)]


In [ ]:
Ks=range(20)
for K in Ks:
    RMSE_val=0.0
    for (u,b,real_r) in val_set:
        u_list=nearest_k_user(u,K)
        if avg_rating[u]==0:
            predict=total_rating_train
        else:
            predict=avg_rating[u]
        for un in u_list:
            if y[un,b]==1:
                predict=ub_rating[(un,b)]/avg_rating[un]*avg_rating[u]
                break
        RMSE_val+=(predict-real_r)**2
    RMSE_val/=len(val_set)
    RMSE_val=math.sqrt(RMSE_val)
    print('K =',K,'RMSE =',RMSE_val)

K = 0 RMSE = 0.8095648163426826
K = 1 RMSE = 0.8157655614251188
K = 2 RMSE = 0.8208545059776513
K = 3 RMSE = 0.824403791086998
K = 4 RMSE = 0.8278764086632061
K = 5 RMSE = 0.8312145367098952
K = 6 RMSE = 0.8343009287485433
K = 7 RMSE = 0.8372386442694089
K = 8 RMSE = 0.8391549692093969
K = 9 RMSE = 0.8434050974668083
K = 10 RMSE = 0.8458266817645542
K = 11 RMSE = 0.8483418987071402
K = 12 RMSE = 0.8507657115545777
K = 13 RMSE = 0.8532712304428333
K = 14 RMSE = 0.8555986286085004
K = 15 RMSE = 0.8583708504541001
K = 16 RMSE = 0.8612801800722036
K = 17 RMSE = 0.8636501688305736
